In [ ]:
from queue import PriorityQueue
import math
from typing import Tuple, List, Dict

In [ ]:
with open("input.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [ ]:
city = lines

In [ ]:
# Well, this was something
# Idea is to use dijkstra-like path finding
# Nodes however are not just their position but also the directions it took to get to them
# so e.g. a tuple ((5,2), ">>>>") means the node at (5,2) and it was reached by going right four times
# This explodes the search space but it still runs in okay time

In [ ]:
def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def equal_dirs(dirs):
    return dirs == len(dirs) * dirs[0]


# Compute a list of all reachable neighbours according to the rules
# Returns list of tuples with dist and neighbour (again being a tuple of position and way to get there)
def get_neighbour_dists(
    node: Tuple[Tuple[int, int], str],
    dists: Dict[Tuple[Tuple[int, int], str], int],
    city: List[str],
) -> List[Tuple[float, Tuple[Tuple[int, int], str]]]:
    nd = node[0]
    nd_hist = node[1]

    dist_to_node = math.inf if node not in dists else dists[node]
    # This should never happen and only catches some manual test cases
    if dist_to_node == math.inf:
        return []

    neighbours = [
        ((nd[0] + 1, nd[1]), "v"),
        ((nd[0] - 1, nd[1]), "^"),
        ((nd[0], nd[1] + 1), ">"),
        ((nd[0], nd[1] - 1), "<"),
    ]

    neighbour_dists = []
    for n_d in neighbours:
        neigh = n_d[0]
        dir = n_d[1]
        # Kick out neighbours that are out of bounds
        if not in_bounds(neigh, city):
            continue

        new_dist = dist_to_node + int(city[neigh[0]][neigh[1]])

        # It is impossible to do 180 turns
        if len(nd_hist) > 0 and (
            ((nd_hist[-1] == ">") and (dir == "<"))
            or ((nd_hist[-1] == "<") and (dir == ">"))
            or ((nd_hist[-1] == "^") and (dir == "v"))
            or ((nd_hist[-1] == "v") and (dir == "^"))
        ):
            continue

        # It mandatory to turn by 90 degrees after 3 times moving in a straight line
        if len(nd_hist) >= 3 and equal_dirs(nd_hist):
            if (
                (nd_hist[0] == ">" or nd_hist[0] == "<") and (dir == ">" or dir == "<")
            ) or (
                (nd_hist[0] == "^" or nd_hist[0] == "v") and (dir == "^" or dir == "v")
            ):
                continue

        new_nd_hist = nd_hist
        if len(nd_hist) >= 3:
            new_nd_hist = nd_hist[1:]

        neighbour_dists.append((new_dist, (neigh, new_nd_hist + dir)))
    # print(f"{node} - {dist_to_node} - {neighbour_dists}")
    return neighbour_dists


def get_path(start, target, predecesors, dists):
    path = []
    pred = [k for k, v in dists.items() if k[0] == target][0]
    while start != pred:
        path.append(pred)
        pred = predecesors[pred]
    path.append(start)
    return list(reversed(path))


def print_path(path, city):
    path = [p[0] for p in path]
    visited_positions = set(path)
    for i in range(0, len(city)):
        for j in range(0, len(city[0])):
            if (i, j) in visited_positions:
                print(".", end="")
            else:
                print(city[i][j], end="")
        print()

In [ ]:
start_node = (0, 0)
target_node = (len(city) - 1, len(city[0]) - 1)
dists = {}
dists[((0, 0), "")] = 0
done_nodes = set()
predecesor = {}
# holds (dist, (node, last_node_dirs))
prio_queue = PriorityQueue()
prio_queue.put((0, (start_node, "")))

while not prio_queue.empty():
    node = prio_queue.get()

    if node[1] in done_nodes:
        continue

    neighbour_dists = get_neighbour_dists(node[1], dists, city)

    for neigh in neighbour_dists:
        old_neigh_dist = dists.get(neigh[1], math.inf)
        if neigh[0] < old_neigh_dist:
            predecesor[neigh[1]] = node[1]
            dists[neigh[1]] = neigh[0]
        prio_queue.put(neigh)

    done_nodes.add(node[1])
    # First reach of target means we are done searching
    # Print node and part of that is distance which in this case is the answer
    if node[1][0] == target_node:
        print(node)
        break

print([f"{k},{v}" for k, v in dists.items() if k[0] == target_node])
print_path(get_path((start_node, ""), target_node, predecesor, dists), city)

part 2

In [ ]:
def get_neighbour_dists_ultra(
    node: Tuple[Tuple[int, int], str],
    dists,
    city: List[str],
) -> List[Tuple[float, Tuple[Tuple[int, int], str]]]:
    nd = node[0]
    nd_hist = node[1]

    dist_to_node = math.inf if node not in dists else dists[node]
    if dist_to_node == math.inf:
        return []

    neighbours = [
        ((nd[0] + 1, nd[1]), "v"),
        ((nd[0] - 1, nd[1]), "^"),
        ((nd[0], nd[1] + 1), ">"),
        ((nd[0], nd[1] - 1), "<"),
    ]

    neighbour_dists = []
    for n_d in neighbours:
        neigh = n_d[0]
        dir = n_d[1]
        if not in_bounds(neigh, city):
            continue

        new_dist = dist_to_node + int(city[neigh[0]][neigh[1]])

        # Need to move at least 4 tiles in one dir before turning
        if len(nd_hist) > 0 and (len(nd_hist) < 4 or not equal_dirs(nd_hist[-4:])):
            if nd_hist[-1] != dir:
                continue

        # also required when reaching end
        if neigh == target_node and not equal_dirs(nd_hist[-3:] + dir):
            continue

        if len(nd_hist) > 0 and (
            ((nd_hist[-1] == ">") and (dir == "<"))
            or ((nd_hist[-1] == "<") and (dir == ">"))
            or ((nd_hist[-1] == "^") and (dir == "v"))
            or ((nd_hist[-1] == "v") and (dir == "^"))
        ):
            continue

        # increse from 3 to 10 that can be moved in one go
        if len(nd_hist) >= 10 and equal_dirs(nd_hist):
            if (
                (nd_hist[0] == ">" or nd_hist[0] == "<") and (dir == ">" or dir == "<")
            ) or (
                (nd_hist[0] == "^" or nd_hist[0] == "v") and (dir == "^" or dir == "v")
            ):
                continue

        new_nd_hist = nd_hist
        if len(nd_hist) >= 10:
            new_nd_hist = nd_hist[1:]

        neighbour_dists.append((new_dist, (neigh, new_nd_hist + dir)))
    # print(f"{node} - {dist_to_node} - {neighbour_dists}")
    return neighbour_dists


start_node = (0, 0)
target_node = (len(city) - 1, len(city[0]) - 1)
dists = {}
dists[((0, 0), "")] = 0
done_nodes = set()
predecesor = {}
# holds (dist, (node, last_node_dirs))
prio_queue = PriorityQueue()
prio_queue.put((0, (start_node, "")))

while not prio_queue.empty():
    node = prio_queue.get()

    if node[1] in done_nodes:
        continue

    neighbour_dists = get_neighbour_dists_ultra(node[1], dists, city)

    for neigh in neighbour_dists:
        old_neigh_dist = dists.get(neigh[1], math.inf)
        if neigh[0] < old_neigh_dist:
            predecesor[neigh[1]] = node[1]
            dists[neigh[1]] = neigh[0]
        prio_queue.put(neigh)

    done_nodes.add(node[1])
    if node[1][0] == target_node:
        print(node)
        break


# print([f"{k},{v}" for k, v in dists.items() if k[0] == target_node])
print_path(get_path((start_node, ""), target_node, predecesor, dists), city)